In [5]:
import sys
import os

# Add the path where custom_model.py is located
sys.path.insert(0, '/home/jupyter-1000790/ML-AI/Project_6 - Tetris/tetris_a/src/custom_model.py')

# Or if it's in a subdirectory like tetris_a/src:
notebook_dir = os.getcwd()
tetris_src_path = os.path.join(notebook_dir, "tetris_a", "src")
sys.path.insert(0, tetris_src_path)


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Frame the problem
Using the customer description, Define the problem your trying to solve in your own words (remember this is not technial but must be specific so the customer understands the project

In [6]:
"""The purpose of this project is to make a self-playing Tetris game that is managed by a model that attempts to survive as long as possibele. Using six different tetris pieces, each with four different possible rientations, my objective is to clear out horizontal lines before letting the stack of blocks hit a certain height."""

'The purpose of this project is to make a self-playing Tetris game that is managed by a model that attempts to survive as long as possibele. Using six different tetris pieces, each with four different possible rientations, my objective is to clear out horizontal lines before letting the stack of blocks hit a certain height.'

# 2. Get the Data 
Define how you recieved the data (provided, gathered..)

In [7]:
"""Ultimately, this project differs greatly from others in many senses of the typicaly ML workflow. In thsi case, I am using reinforcement learning, so the data that I have is necessarily the times that I end up running a certain type of model. For now, the only data that I have access to are a few models that have been provided to me in the form of the already created Tetris game, making it so that I didn't have to code the whole framework myself for the GUI."""

"Ultimately, this project differs greatly from others in many senses of the typicaly ML workflow. In thsi case, I am using reinforcement learning, so the data that I have is necessarily the times that I end up running a certain type of model. For now, the only data that I have access to are a few models that have been provided to me in the form of the already created Tetris game, making it so that I didn't have to code the whole framework myself for the GUI."

# 3. Explore the Data
Gain insights into the data you have from step 2, making sure to identify any bias

In [8]:
"""I can't necessarily explore the given "dataset" per se, but I can still try to play the game to see what works and doesn't work best. By understanding this, I can see what types of features make a good model in comparison to a poorly performing one. So far, the repository we were given has 4 different models to test. A genetic algorithm which performs poorly, a random algorithm which also performs badly, a monte carlo tree search algorithm which doesn't perform that great, but finally, a greedy algorithm which performs very well. After trying these different models on the game, and visualizing them in Pycharm, I came to the conclusion that the files of board.py managed the board, piece.py managed the falling piece, and game.py tied everything together."""

'I can\'t necessarily explore the given "dataset" per se, but I can still try to play the game to see what works and doesn\'t work best. By understanding this, I can see what types of features make a good model in comparison to a poorly performing one. So far, the repository we were given has 4 different models to test. A genetic algorithm which performs poorly, a random algorithm which also performs badly, a monte carlo tree search algorithm which doesn\'t perform that great, but finally, a greedy algorithm which performs very well. After trying these different models on the game, and visualizing them in Pycharm, I came to the conclusion that the files of board.py managed the board, piece.py managed the falling piece, and game.py tied everything together.'

# 4.Prepare the Data


Apply any data transformations and explain what and why


In [9]:
"""There isn't necessarily any data to be preprocessed, because the outcomes of the games or the gameplay itself isn't stored in a database. Once I actually begin modeling, though, using RL, I will have some data that can be managed."""

"There isn't necessarily any data to be preprocessed, because the outcomes of the games or the gameplay itself isn't stored in a database. Once I actually begin modeling, though, using RL, I will have some data that can be managed."

# 5. Model the data
Using selected ML models, experment with your choices and describe your findings. Finish by selecting a Model to continue with


In [18]:
"""I first thought that I should try a RL model, but at first the way that I constructed it, was that my rewards were based on different parameters. I then drew inspiration from both the greedy and genetic models, and figured out the specific criteria like bumpiness and holes which I should apply rewards to. After doing so, the model improved from not learning at all, to being able to advance to around 1000 lines cleared on average after training for a few hours. Ultimately, I decided to continue with this model, and saved it to a pickle file, since the way that I conducted my training, is that it saved the best model through each generation to a pickle file. This final model embodied a specific type of model which is an evolutionary heuristic model based on different weights, which adapt to model efficiency generation by generation."""

'I first thought that I should try a RL model, but at first the way that I constructed it, was that my rewards were based on different parameters. I then drew inspiration from both the greedy and genetic models, and figured out the specific criteria like bumpiness and holes which I should apply rewards to. After doing so, the model improved from not learning at all, to being able to advance to around 1000 lines cleared on average after training for a few hours. Ultimately, I decided to continue with this model, and saved it to a pickle file, since the way that I conducted my training, is that it saved the best model through each generation to a pickle file. This final model embodied a specific type of model which is an evolutionary heuristic model based on different weights, which adapt to model efficiency generation by generation.'

# 6. Fine Tune the Model

With the select model descibe the steps taken to acheve the best rusults possiable 


In [11]:
import numpy as np
import random
from copy import deepcopy

from genetic_helpers import (
    get_peaks, get_holes, get_wells,
    get_bumpiness, get_row_transition, get_col_transition, bool_to_np
)
from board import Board
from piece import Piece, BODIES


class HybridTetrisAgent:
    def __init__(
        self,
        num_features=9,
        alpha=0.02,  # slightly higher learning rate
        gamma=0.97,  # slightly higher discount
        epsilon=1.0,
        epsilon_min=0.05,
        epsilon_decay=0.9995  # slower decay for more exploration early
    ):
        self.weights = np.random.uniform(-0.01, 0.01, num_features)
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay

    def extract_features(self, board_np):
        peaks = get_peaks(board_np)
        holes = get_holes(peaks, board_np)
        wells = get_wells(peaks)

        # normalize features to roughly similar scale
        peaks_sum = np.sum(peaks) / 20
        holes_sum = np.sum(holes) / 10
        bumpiness = get_bumpiness(peaks) / 10
        empty_cols = np.count_nonzero(np.count_nonzero(board_np, axis=0) == 0) / board_np.shape[1]
        max_wells = np.max(wells) / 10
        cols_with_holes = np.count_nonzero(np.array(holes) > 0) / board_np.shape[1]
        row_trans = get_row_transition(board_np, np.max(peaks)) / board_np.shape[1]
        col_trans = get_col_transition(board_np, peaks) / board_np.shape[1]
        rows_filled = np.sum(np.count_nonzero(board_np, axis=1)) / board_np.shape[0]

        return np.array([
            peaks_sum,
            holes_sum,
            bumpiness,
            empty_cols,
            max_wells,
            cols_with_holes,
            row_trans,
            col_trans,
            rows_filled
        ], dtype=float)

    def q_value(self, features):
        return float(np.dot(self.weights, features))

    def choose_action(self, board, piece, lookahead=2):
        # epsilon-greedy
        if random.random() < self.epsilon:
            rot_count = random.randint(0, 3)
            temp_piece = piece
            for _ in range(rot_count):
                temp_piece = temp_piece.get_next_rotation()
            max_x = board.width - len(temp_piece.skirt)
            x = random.randint(0, max_x)
            return x, temp_piece

        best_value = -1e18
        best_x = None
        best_piece = None

        rot_piece = piece
        for _ in range(4):
            rot_piece = rot_piece.get_next_rotation()
            max_x = board.width - len(rot_piece.skirt)
            for x in range(max_x + 1):
                try:
                    y = board.drop_height(rot_piece, x)
                except:
                    continue

                # simulate placement
                grid = deepcopy(board.board)
                for pos in rot_piece.body:
                    grid[y + pos[1]][x + pos[0]] = True

                # lookahead: average over a few random next pieces
                lookahead_value = 0
                for _ in range(lookahead):
                    next_piece = Piece(body=random.choice(BODIES)[0])
                    next_rot = next_piece.get_next_rotation()
                    max_x2 = board.width - len(next_rot.skirt)
                    x2 = random.randint(0, max_x2)
                    y2 = 0
                    try:
                        y2 = board.drop_height(next_rot, x2)
                    except:
                        continue
                    grid2 = deepcopy(grid)
                    for pos in next_rot.body:
                        grid2[y2 + pos[1]][x2 + pos[0]] = True
                    f2 = self.extract_features(bool_to_np(grid2))
                    lookahead_value += self.q_value(f2)
                lookahead_value /= lookahead

                f = self.extract_features(bool_to_np(grid))
                v = self.q_value(f) + 0.5 * lookahead_value  # weight lookahead

                if v > best_value:
                    best_value = v
                    best_x = x
                    best_piece = rot_piece

        return best_x, best_piece

    def update(self, prev_features, reward, next_features):
        reward = np.clip(reward, -1, 1)
        prev_q = self.q_value(prev_features)
        next_q = self.q_value(next_features)
        td_target = reward + self.gamma * next_q
        td_error = td_target - prev_q
        td_error = np.clip(td_error, -10, 10)  # allow slightly bigger updates
        self.weights += self.alpha * td_error * prev_features
        self.weights = np.nan_to_num(self.weights)
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)


def train_hybrid_agent(episodes=1000):
    agent = HybridTetrisAgent()
    print("Starting Hybrid Tetris training...")

    best_moves = 0
    last50_moves = []

    for episode in range(1, episodes + 1):
        board = Board()
        total_reward = 0
        moves = 0
        done = False

        while not done:
            piece = Piece(body=random.choice(BODIES)[0])
            prev_board_np = bool_to_np(board.board)
            prev_features = agent.extract_features(prev_board_np)

            x, piece_used = agent.choose_action(board, piece)
            if x is None or piece_used is None:
                done = True
                break

            try:
                y = board.drop_height(piece_used, x)
                board.place(x, y, piece_used)
            except:
                done = True
                break

            cleared = board.clear_rows()
            next_board_np = bool_to_np(board.board)
            next_features = agent.extract_features(next_board_np)

            holes_next = np.sum(get_holes(get_peaks(next_board_np), next_board_np))
            height_penalty = np.max(get_peaks(next_board_np)) / 20

            reward = cleared * 1.0 - holes_next * 0.05 - height_penalty * 0.02

            total_reward += reward
            agent.update(prev_features, reward, next_features)
            moves += 1

            if np.any(board.heights >= board.height):
                done = True

        last50_moves.append(moves)
        if len(last50_moves) > 50:
            last50_moves.pop(0)
        avg_last50 = np.mean(last50_moves)
        best_moves = max(best_moves, moves)

        print(
            f"Gen {episode:4d}/{episodes} | "
            f"Moves: {moves:4d} | "
            f"Best: {best_moves:4d} | "
            f"Avg50: {avg_last50:6.2f} | "
            f"Reward: {total_reward:7.3f} | "
            f"Eps: {agent.epsilon:.3f}"
        )

    np.save("trained_weights.npy", agent.weights)
    print("Training complete! Saved weights to trained_weights.npy")
    return agent


if __name__ == "__main__":
    train_hybrid_agent(episodes=1000)




'
"""I improved the model by slightly increasing alpha for faster learning and raising gamma to value long-term rewards. Epsilon decays slower to encourage early exploration, while feature normalization stabilizes updates. A lookahead mechanism evaluates candidate placements with a few upcoming pieces for smarter decisions, and reward shaping—based on cleared lines, holes, and max height—guides the agent toward more effective play."""

SyntaxError: unterminated string literal (detected at line 204) (4012437259.py, line 204)

# 7. Present
In a customer faceing Document provide summery of finding and detail approach taken


In [19]:
"""Ultimately, this project had a specific approach which revolved around one main idea. Essentially, through the use of reinforcement learning, I 
planned to make my model learn as I trained it. So far, it has continued to be trained, and its accuracy in playing the tetris game continues to improve. 
Specificaly, though, I used some of the greedy and genetic models which had certain benefits, mixed them together, and created a product which was
better than before. The genetic model made it so that the model could keep getting better, whlie greedy was already good from the get go. I then fine
tuned this model, so that the epsilon value would decraese slower, and the testing would be conducted in a better way. I concluded that the evolutionary 
model with heuristic weights actually worked the best, because in comparison to the first RL model I tried, the model actually learned from its mistakes,
and the rewards that were given improved its efficiency."""

'Ultimately, this project had a specific approach which revolved around one main idea. Essentially, through the use of reinforcement learning, I \nplanned to make my model learn as I trained it. So far, it has continued to be trained, and its accuracy in playing the tetris game continues to improve. \nSpecificaly, though, I used some of the greedy and genetic models which had certain benefits, mixed them together, and created a product which was\nbetter than before. The genetic model made it so that the model could keep getting better, whlie greedy was already good from the get go. I then fine\ntuned this model, so that the epsilon value would decraese slower, and the testing would be conducted in a better way. I concluded that the evolutionary \nmodel with heuristic weights actually worked the best, because in comparison to the first RL model I tried, the model actually learned from its mistakes,\nand the rewards that were given improved its efficiency.'

# 8. Launch the Model System
Define your production run code, This should be self susficent and require only your model pramaters 


In [ ]:
def inference(checkpoint_file="best_agent.pkl", max_pieces=100000, print_progress=True):
    
    import pickle
    from custom_model import EvolutionaryAgent
    from board import Board
    from piece import Piece
   
    try:
        with open(checkpoint_file, "rb") as f:
            trained_agent = pickle.load(f)
        print("Loaded", checkpoint_file)
        print(f"Trained weights: {trained_agent.weights}")
    except FileNotFoundError:
        print(f"No {checkpoint_file} found - make sure you ran training first!")
        return None, None, None
    
    print("\n" + "="*60)
    print("Testing trained agent...")
    print("="*60)
    
    board = Board()
    pieces_placed = 0
    lines_cleared = 0
    
    while pieces_placed < max_pieces:
        piece = Piece()
        x, piece = trained_agent.get_best_move(board, piece)
        
        if x is None:
            break
        
        y = board.drop_height(piece, x)
        
        try:
            board.place(x, y, piece)
        except:
            break
        
        cleared = board.clear_rows()
        lines_cleared += cleared
        pieces_placed += 1
        
        if print_progress and pieces_placed % 100 == 0:
            print(f"Progress: {pieces_placed} pieces, {lines_cleared} lines cleared")
        
        if board.top_filled():
            break
    
    score = pieces_placed + (lines_cleared * 10)
    
    print(f"\nGame complete!")
    print(f"Pieces placed: {pieces_placed}")
    print(f"Lines cleared: {lines_cleared}")
    print(f"Score: {score:.1f}")
    print("="*60)
    
    return score, pieces_placed, lines_cleared

score, pieces, lines = inference("best_agent_fixed.pkl")

Loaded best_agent_fixed.pkl
Trained weights: [ 0.81871496  1.58361802  0.38366396 -0.68594702]

Testing trained agent...
Progress: 100 pieces, 42 lines cleared
Progress: 200 pieces, 83 lines cleared
Progress: 300 pieces, 125 lines cleared
Progress: 400 pieces, 166 lines cleared
Progress: 500 pieces, 209 lines cleared
Progress: 600 pieces, 245 lines cleared
Progress: 700 pieces, 295 lines cleared
Progress: 800 pieces, 338 lines cleared
Progress: 900 pieces, 379 lines cleared
Progress: 1000 pieces, 419 lines cleared
Progress: 1100 pieces, 466 lines cleared
Progress: 1200 pieces, 505 lines cleared
Progress: 1300 pieces, 547 lines cleared
Progress: 1400 pieces, 589 lines cleared
Progress: 1500 pieces, 629 lines cleared
Progress: 1600 pieces, 670 lines cleared
Progress: 1700 pieces, 717 lines cleared
Progress: 1800 pieces, 758 lines cleared
Progress: 1900 pieces, 805 lines cleared
Progress: 2000 pieces, 846 lines cleared
Progress: 2100 pieces, 885 lines cleared
Progress: 2200 pieces, 924 li